### Creating a simple neural network to recognise hand-written digits

Importing data manipulation libraries

In [15]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

Importing deep learning library

In [16]:
from tensorflow import keras; print('Keras version: ', keras.__version__)

Keras version:  2.4.0


Reading in training data

In [17]:
digits_train = pd.read_csv('../input/digit-recognizer/train.csv')
digits_train.head(n = 5)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Separating the labels and the predictors

In [50]:
targets_train = pd.get_dummies(data = digits_train.label)
targets_train.head()

preds_train = digits_train.drop(labels = 'label', axis = 1)
preds_train.head()
preds_train.iloc[:, 350:370]

,pixel350,pixel351,pixel352,pixel353,pixel354,pixel355,pixel356,pixel357,pixel358,pixel359,pixel360,pixel361,pixel362,pixel363,pixel364,pixel365,pixel366,pixel367,pixel368,pixel369
0,253,254,240,81,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,25,240,254,254,153,0,0,0,0,0,0,0,0,0,0,64
2,254,184,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,195,227,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,26,200,253,253,253,253,5,0,0,0,0,0,0,0,0,254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,115,254,150,0,0,0,0,0,0,0,0,0,41
41996,253,254,229,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41997,0,0,0,0,191,255,255,128,0,0,0,0,0,0,0,0,0,0,0,64
41998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Standardising the predictor values

In [52]:
from sklearn.preprocessing import StandardScaler

std_preds_train = pd.DataFrame(StandardScaler().fit_transform(preds_train.iloc[:]))
std_preds_train.iloc[:, 350:370]

,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369
0,1.500243,1.368737,1.145959,-0.336913,-0.970152,-0.793904,-0.614823,-0.470261,-0.356279,-0.257428,-0.156372,-0.054691,-0.021258,-0.011189,-0.00488,-0.009422,-0.026492,-0.057099,-0.123074,-0.239056
1,-0.821316,-0.920364,-1.035552,-1.065357,-0.745218,1.467259,2.060659,2.643810,1.916394,-0.257428,-0.156372,-0.054691,-0.021258,-0.011189,-0.00488,-0.009422,-0.026492,-0.057099,-0.123074,1.190772
2,1.509419,0.737882,-1.035552,-1.065357,-0.970152,-0.793904,-0.614823,-0.470261,-0.356279,-0.257428,-0.156372,-0.054691,-0.021258,-0.011189,-0.00488,-0.009422,-0.026492,-0.057099,-0.123074,-0.239056
3,-0.821316,-0.920364,-1.035552,-1.065357,0.784333,1.344779,-0.614823,-0.470261,-0.356279,-0.257428,-0.156372,-0.054691,-0.021258,-0.011189,-0.00488,-0.009422,-0.026492,-0.057099,-0.123074,-0.239056
4,-0.821316,-0.920364,-1.035552,-1.065357,-0.736221,1.090399,2.050125,2.631550,3.401802,4.770190,0.018954,-0.054691,-0.021258,-0.011189,-0.00488,-0.009422,-0.026492,-0.057099,-0.123074,5.435571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,-0.821316,-0.920364,-1.035552,-1.065357,-0.970152,-0.793904,-0.614823,0.939653,3.416656,2.723373,-0.156372,-0.054691,-0.021258,-0.011189,-0.00488,-0.009422,-0.026492,-0.057099,-0.123074,0.676927
41996,1.500243,1.368737,1.045973,-1.065357,-0.970152,-0.793904,-0.614823,-0.470261,-0.356279,-0.257428,-0.156372,-0.054691,-0.021258,-0.011189,-0.00488,-0.009422,-0.026492,-0.057099,-0.123074,-0.239056
41997,-0.821316,-0.920364,-1.035552,-1.065357,0.748344,1.608582,2.071192,1.099035,-0.356279,-0.257428,-0.156372,-0.054691,-0.021258,-0.011189,-0.00488,-0.009422,-0.026492,-0.057099,-0.123074,1.190772
41998,-0.821316,-0.920364,-1.035552,-1.065357,-0.970152,-0.793904,-0.614823,-0.470261,-0.356279,-0.257428,-0.156372,-0.054691,-0.021258,-0.011189,-0.00488,-0.009422,-0.026492,-0.057099,-0.123074,-0.239056


Defining the model and printing a summary

In [64]:
nn_model = keras.Sequential([
    keras.layers.Dense(1024, input_shape = (preds_train.values.shape[1],), 
                       activation = 'sigmoid'),
    keras.layers.Dense(128, activation = 'sigmoid'),
    #    keras.layers.Dropout(0.2), # Could be added if the fit metrics indicated overfitting
    keras.layers.Dense(64, activation = 'sigmoid'),
#    keras.layers.Dense(64, activation = 'sigmoid'),
#    keras.layers.Dense(32, activation = 'sigmoid'),
    keras.layers.Dense(len(targets_train.columns), activation = 'softmax')
])

nn_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 1024)              803840    
_________________________________________________________________
dense_21 (Dense)             (None, 128)               131200    
_________________________________________________________________
dense_22 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_23 (Dense)             (None, 10)                650       
Total params: 943,946
Trainable params: 943,946
Non-trainable params: 0
_________________________________________________________________


Compiling the model

In [71]:
nn_model.compile(optimizer = keras.optimizers.Adam(learning_rate = 0.0001),
                 loss = keras.losses.CategoricalCrossentropy(),
                 metrics = ['accuracy'])

Fitting the model

In [72]:
nn_model.fit(x = std_preds_train.values,
             y = targets_train.values,
             #batch_size = 1,
             epochs = 20)

Epoch 1/20
1313/1313 [==============================] - 3s 3ms/step - loss: 8.8929e-06 - accuracy: 1.0000
Epoch 2/20
1313/1313 [==============================] - 3s 2ms/step - loss: 6.8958e-06 - accuracy: 1.0000
Epoch 3/20
1313/1313 [==============================] - 3s 2ms/step - loss: 5.9318e-06 - accuracy: 1.0000
Epoch 4/20
1313/1313 [==============================] - 4s 3ms/step - loss: 5.2482e-06 - accuracy: 1.0000
Epoch 5/20
1313/1313 [==============================] - 3s 2ms/step - loss: 4.7300e-06 - accuracy: 1.0000
Epoch 6/20
1313/1313 [==============================] - 4s 3ms/step - loss: 4.3048e-06 - accuracy: 1.0000
Epoch 7/20
1313/1313 [==============================] - 3s 3ms/step - loss: 3.9775e-06 - accuracy: 1.0000
Epoch 8/20
1313/1313 [==============================] - 4s 3ms/step - loss: 3.6306e-06 - accuracy: 1.0000
Epoch 9/20
1313/1313 [==============================] - 3s 3ms/step - loss: 3.3841e-06 - accuracy: 1.0000
Epoch 10/20
1313/1313 [=======================

Saving the model

In [73]:
nn_model.save('MNIST_nn_model.h5')

Reading in the test set

In [74]:
digits_test = pd.read_csv('../input/digit-recognizer/test.csv')
digits_test.head(n = 5)

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Applying the sam standardisation step

In [75]:
std_digits_test = pd.DataFrame(StandardScaler().fit_transform(digits_test.iloc[:]))

Predicting the labels using the model

In [76]:
test_predictions = nn_model.predict(std_digits_test.values)

test_predictions

array([[9.2963419e-13, 4.5465867e-09, 1.0000000e+00, ..., 1.0007587e-09,
        1.4340565e-09, 2.5517347e-11],
       [9.9999964e-01, 2.4046994e-13, 7.3115441e-10, ..., 5.1549481e-15,
        7.4661076e-11, 1.8238158e-11],
       [1.6397620e-16, 5.5868311e-15, 2.1519522e-15, ..., 6.7025687e-09,
        4.2297063e-10, 9.9999905e-01],
       ...,
       [2.0483658e-20, 2.1982817e-11, 1.2068195e-11, ..., 2.9144827e-08,
        9.8555109e-11, 6.4678605e-09],
       [9.3603882e-18, 8.6613420e-17, 4.5785429e-16, ..., 4.0201393e-09,
        4.2673547e-12, 1.0000000e+00],
       [1.3178210e-12, 4.0864392e-09, 1.0000000e+00, ..., 1.0119298e-09,
        9.3034336e-10, 9.3182045e-11]], dtype=float32)

Collecting the predictions and image IDs in a data frame

In [77]:
predictions = []; id = []

for i in range(len(test_predictions)):
        predictions.append(np.argmax(test_predictions[i]))
        id.append(i+1)
        
predictions_df = pd.concat([pd.DataFrame(id, columns = ['ImageId']), 
                            pd.DataFrame(predictions, columns = ['Label'])], 
                           axis = 1)

Saving the results as a .csv for submission

In [79]:
predictions_df.to_csv('predictions.csv', index = False)